In [21]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 10,
  'palette': ['000000', 'F5F5F5']}



In [ ]:
import os

import ee
from ssebop_stats import ssebop, ee_manager

images = {}

for year in (2022, ):  # range(2015, 2023):
    runner = ssebop.SSEBOPer()
    runner.export_folder = f"year_comparison_{year}"
    runner.comparison(year, "03-01", "05-31", "06-01", "07-31", method="divide")
    images[year] = runner
    
runner_march_april = ssebop.SSEBOPer()
runner_march_april.pixel_reducer = "min"
runner_march_april.run(2022, "03-01", "04-30")

runner_june_july = ssebop.SSEBOPer()
runner_june_july.pixel_reducer = "max"
runner_june_july.run(2022, "06-01", "07-31")'

In [25]:
# Create a folium map object.
my_map = folium.Map(location=[35, -120], zoom_start=8)

# Add the elevation model to the map object.
my_map.add_ee_layer(images[2022].results, vis_params, 'GEE ET Ratio 2022')
my_map.add_ee_layer(runner_march_april.results, vis_params, 'Spring 2022 Min ET')
my_map.add_ee_layer(runner_june_july.results, vis_params, 'Summer 2022 Max ET')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)